# Sierra REST API -- Bulk Item-level Holds 

The purpose of this script is to place item-level holds for given patron records / accounts from a given Excel sheet having 2 columns:

1. `item_barcode` : the barcode from the item
1. `branch_account` : the patron record number associated with the branch account (note that the record number should contain the check digit)

The input Excel Workbook should consist of 1 sheet, and have 2 columns as defined above. Below is an example of how it would look (with 2 rows of example data provided):

| barcode       	| branch_account 	|
|---------------	|----------------	|
| A000055325526 	| p10023768      	|
| A000057896540 	| p10023768      	|
| A000067386443 	| p10023768      	|
| A000071676870 	| p10023768      	|

## Getting Started / Setting Up the Environment

This script has some dependencies and requirements in order to run successfully.

Follow the steps outlined below

1. Run the following cell which will set up the environment for the script.
   
   **NOTE:** If you are prompted to "RESTART RUNTME", click on that button and proceed to the next cell

In [ ]:
# click the play button to the left, restart the runtime if needed

# point this to the base URL for the Sierra REST API
base_url = 'https://sierra-test.cincinnatilibrary.org:443/iii/sierra-api/v6/'

!pip install -U pip
!pip install pandas==1.3.4 openpyxl==3.0.9 requests==2.26.0 SQLAlchemy==1.4.27

# import the needed modules
from base64 import b64encode
from getpass import getpass
from google.colab import files
from io import BytesIO
import json
import numpy as np
import pandas as pd
import re
import requests
from sqlalchemy import create_engine

## Enter REST-API Credentials

In order to be authorized to use the REST API, you must enter valid credientials.

When prompted ...

1. Input client **KEY** value, press enter
1. Input client **SECRET** value, press enter

In [ ]:
client_key = getpass('Enter the client KEY value: ')
client_secret = getpass('Enter the client SECRET value: ')

In [ ]:
# this will get an initial access token
def set_access_headers():
    """
    use this function to set and refresh the access_headers for future
    authorizing API requests
    """

    auth_string = b64encode(
      (client_key + ':' + client_secret).encode('utf-8')
    ).decode('utf-8')

    headers = {}
    headers['authorization'] = 'basic ' + auth_string

    try:
        r = requests.post(base_url + 'token', headers=headers, verify=True)

    except requests.ConnectionError as e:
        print('connection error: {}'.format(e))
        return 0

    if r.status_code != 200:
        print(r.status_code)
        return 0

    access_token = r.json()['access_token']

    # set our headers to use the access token
    headers['authorization'] = 'bearer ' + access_token

    # Note: depending on the Sierra REST API request endpoint,
    # you may need to change the types below to fit the request,
    # but these are pretty standard
    headers['content-type'] = 'application/json'
    headers['accept'] = 'application/json'

    return headers

# this refreshes the token, and creates the header we need to send in the request
headers = set_access_headers()

# get the response as r
r = requests.get(base_url + 'info/token', headers=headers, verify=True)

# get the how many seconds are left before the token expires ...
print('access token expires in: {} seconds'.format(r.json()['expiresIn']))

## Upload Input File

This file will be processed to placed the holds.

When prompted, upload the input file

In [ ]:
input_file = files.upload()

In [ ]:
df = pd.read_excel( 
    [key for key in input_file.keys()][0], 
    converters={
        'barcode': str,        
    }
)
print(df.head(5))
print('...')
print('total rows in input: ', df.shape[0])

In [ ]:
# construct the item query we'll use to convert barcodes to item record numbers (API endpoint URLs)
operands = [row['barcode'] for i, row in df.iterrows()]
item_query = {"queries": [{"target": {"record": {"type": "item"}, "field": {"tag": "b"}}, "expr": [{"op": "in", "operands": operands}]}]}

print(json.dumps(item_query))

headers = set_access_headers()

print('\nsearching ...')

r = requests.post(
    base_url + 'items/query?offset=0&limit={}'.format(len(operands)), 
    headers=headers,
    data=json.dumps(item_query),
    verify=True
)

print("response json: ", r.json())

print('...')

print('total rows of items from search: {}'.format(r.json()['total']) )

In [ ]:
for i, entry in enumerate(r.json()['entries']):
    print(i, entry['link'])
    r_entry = requests.get(
        url=entry['link'] + '?fields=id,location,status,barcode,bibIds',
        headers=headers,
        verify=True
    )
    print(r_entry.json())
    
    # extract the item record number
    item_record_num = r_entry.json()['id']
        
    #extract the barcode
    search_barcode = str(r_entry.json()['barcode'])
    
    # match the barcode to get the patron record 
    # (remember that we're stripping off extra spaces, the 'p' and the last 'checkdigit'
    patron_record_num = df[df.barcode == search_barcode].branch_account.values[0].strip()[1:-1]
    
    print('placing hold...')
    print(f'item_record_num:\t{item_record_num}\nsearch_barcode:\t\t{search_barcode}\npatron_record_num:\t{patron_record_num}')

    # patron_record_num = df[df.barcode == search_barcode].branch_account.values[0].strip()[1:-1]
    
    # place the hold ...
    
    body_data = {
        "recordType": "i",
        "recordNumber": int(item_record_num),
        "pickupLocation": "1",
        # "neededBy": "",
        # "numberOfCopies": 1,
        "note": "chpl-bulk-hold-script"
    }
    
    r_hold = requests.post(
        url=base_url + f'patrons/{patron_record_num}/holds/requests',
        headers=headers,
        data=json.dumps(body_data),
        verify=True
    )

    print("response json: ", r_hold.json())
    
    print('---')
